In [1]:
#xmlファイルを読んでピクセル座標に変換
#fitsからaperture photometryやっちゃう
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import astropy.wcs
from astropy.io import fits
import astropy.io
from astropy.stats import sigma_clipped_stats
from photutils import datasets
from photutils import DAOStarFinder
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture
from photutils import CircularAnnulus
from scipy.optimize import curve_fit
from photutils import aperture_photometry
from matplotlib.backends.backend_pdf import PdfPages
from IPython.display import clear_output

df = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e8.csv")
#df = pd.read_csv("test8.csv")

############################################ リチェックするフレーム #####
"""
filename_fits2 = "d_fits/sub" + df["ID"] + ".fits"
d_list = ["d_fits/sub665114_014.fits"]
d_idx = np.arange(1).reshape(1) 
for i in d_list:
    for j in range(len(filename_fits2)):
        if i == filename_fits2[j]:
            d_idx = np.vstack((d_idx, j))
d_idx = np.delete(d_idx,0)
print d_idx
#df = df.drop(d_idx,axis=0) #d_listの該当する行だけ消してる
"""
###############################################################
filename_fits = "d_fits/sub" + df["ID"] + ".fits"
filename_fits = "d_fits/sub" + df["ID"] + ".fits"
filename_csv = "d_csv_ucac_modified/sub" + df["ID"] + "_star.csv"
output_csv = "sub" + df["ID"] + "_star_used.csv"
filename_pdf = "sub" + df["ID"] + "_ucac.pdf"
meteor_flux_list = df["flux_meteor"].values
altitude_meteor = df["Beginning_height_km_"].values #absolute magnitude
zenith_angle = df["ZenithOfMeteorStartPoint"].values
azimuth_angle = df["AzimuthOfMeteorStartPoint"].values

earth_radius = 6.3781e3#[km]
kiso_longitude_deg = 137.62535 # Kiso_observatory
kiso_latitude_deg = 35.7973
mu_longitude_deg = 136.1089 # MU_observatory
mu_latitude_deg = 34.8522

X_kiso = earth_radius*np.sin(np.deg2rad(90. - kiso_latitude_deg))*np.cos(np.deg2rad(kiso_longitude_deg))
Y_kiso = earth_radius*np.sin(np.deg2rad(90. - kiso_latitude_deg))*np.sin(np.deg2rad(kiso_longitude_deg))
Z_kiso = earth_radius*np.cos(np.deg2rad(90. - kiso_latitude_deg))
X_mu = earth_radius*np.sin(np.deg2rad(90. - mu_latitude_deg))*np.cos(np.deg2rad(mu_longitude_deg))
Y_mu = earth_radius*np.sin(np.deg2rad(90. - mu_latitude_deg))*np.sin(np.deg2rad(mu_longitude_deg))
Z_mu = earth_radius*np.cos(np.deg2rad(90. - mu_latitude_deg))
z_meteor = altitude_meteor
radius_meteor = z_meteor/np.cos(np.deg2rad(zenith_angle))
x_meteor = radius_meteor*np.sin(np.deg2rad(zenith_angle))*np.cos(np.deg2rad(azimuth_angle))
y_meteor = radius_meteor*np.sin(np.deg2rad(zenith_angle))*np.sin(np.deg2rad(azimuth_angle))
X_meteor = X_mu + x_meteor
Y_meteor = Y_mu + y_meteor
Z_meteor = Z_mu + z_meteor
distance = np.power((X_meteor-X_kiso),2) + np.power((Y_meteor-Y_kiso),2) + np.power((Z_meteor-Z_kiso),2)
l = np.sqrt(distance)

#distance = 170. #[km]
#alt = np.power(altitude_meteor, 2)
#distance = np.power(distance, 2)
#l = alt + distance
#l = l.astype(np.float64)
#l = np.sqrt(l)

###############################################################
def gauss(x, A, mu, sigma):
    return A / np.sqrt(2.0*np.pi) / sigma * np.exp(-((x-mu)/sigma)**2/2)

def pogson(x, a):
    return -2.5*np.log10(x) + a

photometry_table = np.arange(6).reshape(1,6)
for idx in range(len(filename_fits)):
#for idx in range(0,20):
    meteor_flux = meteor_flux_list[idx]
    print filename_fits[idx],idx
    fits = astropy.io.fits.open(filename_fits[idx])
    hdul = fits
    data = hdul[0].data
    data = data[1,:,:]#差分前のフレームだけにする
    data = np.reshape(data, (282, 500))#2次元化
    
###################################### csv ファイルの読み込み #######
    df_gaia = pd.read_csv(filename_csv[idx])
    source_ID = df_gaia["sourceID"].values
    g_mag_star = df_gaia["VMag"].values
    x = df_gaia["x_astropy"].values
    x = x.astype(np.int64)
    y = df_gaia["y_astropy"].values
    y = y.astype(np.int64)
    g_mag_star = np.vstack((g_mag_star,x,y))
    g_mag_star = g_mag_star.transpose()#["g_mag", "x", "y"]
    g_mag_star = g_mag_star[np.argsort(g_mag_star[:, 0])] #等級でソート
    g_mag_star = np.delete(g_mag_star, np.where(g_mag_star[:,0]<10.), 0)
    g_mag_star = np.delete(g_mag_star, np.where(g_mag_star[:,0]>16.), 0)
    
###############################################################
############ Gaussian profile of catalog stars to estimate aperture's parameters ###
    star_flux = np.arange(1).reshape(1)
    for i in range(len(g_mag_star)):
        #print filename_fits[idx],idx
        #print source_ID[i],"ID_number=",i
        xr1 = g_mag_star[i,1] - 10
        xr2 = g_mag_star[i,1] + 10
        xr1 = xr1.astype(np.int64)  #ここintにする必要ある？
        xr2 = xr2.astype(np.int64)
        xr = range(xr1, xr2, 1)
        yr = g_mag_star[i,2]
        yr = yr.astype(np.int64)
        y =  data[yr,xr]
        A_para = y.max() #ピークの部分
        mu_para = g_mag_star[i,1] #恒星の中心x座標
        p0 = [A_para, mu_para, 1.] # パラメータの初期値(A, mu, sigma)
        parameter,_ = curve_fit(gauss, xr, y, p0=p0)
        sigma = parameter[2]
        fwhm = 2*parameter[2]*(np.sqrt(2*np.log(2)))
        
##################################################################
####################################### Conducting aperture photometry ###
        positions = g_mag_star[i,1:3] #starの(x,y)座標
        aperture_radius = 1.5*fwhm
        inner_radius = aperture_radius + fwhm
        outer_radius = 5*fwhm
        if fwhm > 0:
            apertures = CircularAperture(positions, r=aperture_radius)
            annulus_apertures = CircularAnnulus(positions, r_in=inner_radius, r_out=outer_radius)
            apers = [apertures, annulus_apertures]
            phot_table = aperture_photometry(data, apers)
            for col in phot_table.colnames:
                phot_table[col].info.format = '%.8g'
            bkg_mean = phot_table['aperture_sum_1'] / annulus_apertures.area()
            bkg_sum = bkg_mean * apertures.area()
            final_sum = phot_table['aperture_sum_0'] - bkg_sum
            phot_table['residual_aperture_sum'] = final_sum
            phot_table['residual_aperture_sum'].info.format = '%.8g'
            aperture_sum = np.array(phot_table["residual_aperture_sum"][:])
        else:
            aperture_sum = 0
        star_flux = np.vstack((star_flux, aperture_sum))
        #clear_output()
    
    star_flux = np.delete(star_flux,0, 0)
    star_number = len(g_mag_star) #測光に使った恒星の数
    if len(star_flux) != 0 :# 恒星のfluxが測れてない
        g_mag_star = np.hstack((g_mag_star,star_flux))
        delete_index =np.where(g_mag_star[:,3]<=0.)
        g_mag_star = np.delete(g_mag_star, delete_index, 0)#star_fluxがマイナスのものを消す
        source_ID = np.delete(source_ID,delete_index,0)
        
    #[GMag", "Gmag_err", "x_astropy", "y_astropy", "star_flux"]
    #アパーチャー測光やって恒星のフラックスまで出した
    #g_mag_starとg_mag_star_errとstar_flux使って原点補正
    #さちってる10等級以上を消す
    #パラメーターの初期値は25mag(等級原点)
    #meteor_fluxがゼロの場合、等級なしに
    
        #source_ID = source_ID.reshape(len(source_ID),1)
        #g_mag_star = np.hstack((g_mag_star,source_ID))
        #columns =["v_mag_star","x_astropy","y_astropy", "star_flux","source_ID"] 
        #output_table = pd.DataFrame(data=g_mag_star,columns=columns)
        #output_table.to_csv(output_csv[idx])
        clear_output()
    
        g_mag_star = g_mag_star.astype(np.float64)
        param, cov = curve_fit(pogson, g_mag_star[:,3], g_mag_star[:,0],sigma=g_mag_star[:,1],p0=24.5)
        param_err = np.sqrt(np.diag(cov))
        g_mag_meteor = -2.5*np.log10(meteor_flux) + param # Estimation  G-band magnitude
        g_mag_meteor_err = param_err
        g_band_abs_mag_meteor = g_mag_meteor + 10 - 5*np.log10(l[idx]) #100km換算の絶対等級
        g_band_abs_mag_meteor_error = param_err
    else:
        g_mag_meteor = 100
        g_band_abs_mag_meteor = 100
    if meteor_flux == 0 :#恒星0個 or 流星のfluxが測れてない
        g_mag_meteor = 100
        g_band_abs_mag_meteor = 100
        
    T = np.hstack((g_mag_meteor, g_mag_meteor_err, star_number,\
                   g_band_abs_mag_meteor,g_band_abs_mag_meteor_error,param))
    photometry_table = np.vstack((photometry_table,T))
    #["mag","err","star_num","abs_mad","err"]
##################################################################
###################################################### グラフ作る  ##    
"""
    #print param,cov,param_err
    flux_axis = np.logspace(1, 10, 10, base=10)
    mag_fit = -2.5*np.log10(flux_axis) + param[0]
    title = "star_number = " + str(star_number)
    
    fig = plt.figure(figsize=(10,10))
    plt.subplots_adjust(hspace=0.3)
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.subplot(2,2,1)
    plt.scatter(g_mag_star[:,3], g_mag_star[:,0],color = "red",marker='.', s=5,zorder=3,\
                label='star_flux') #star
    plt.plot(flux_axis, mag_fit, color = "red",linestyle='dashdot',linewidth=0.5\
             ,zorder=1,label='fitting_line')
    #plt.errorbar(g_mag_star[:,3], g_mag_star[:,0],xerr=g_mag_star[:,1],fmt='k|')
    if g_mag_meteor != 100: 
        plt.scatter(meteor_flux, g_mag_meteor,color = "blue",marker='.',s=10,zorder=4,\
                    label='meteor_flux') #meteor
        plt.errorbar(meteor_flux, g_mag_meteor,xerr=g_mag_meteor_err,\
                 fmt='k|')
    plt.xscale('log')
    plt.grid(which='major',color='black',linestyle='dashed',alpha=0.5)
    plt.grid(which='minor',color='black',linestyle='dashed')
    plt.xlabel("Count Flux")
    plt.ylabel("G-band Magnitude")
    plt.title(title,loc='left')
    #plt.legend(prop={'size':6})
    plt.legend()
    
    plt.subplot(2,1,2)
    plt.title(filename_fits[idx])
    plt.imshow(data, cmap = 'gray', vmin = 0, vmax = 400, interpolation = 'none')
    positions = (g_mag_star[:,1], g_mag_star[:,2])
    apertures = CircularAperture(positions, r=4.)
    apertures.plot(color='green', lw=1.5, alpha=0.5)
    #star_index = np.arange(star_number) + 1
    for number in range(len(g_mag_star)):
        star_index = "star_" + str(number)
        plt.text(g_mag_star[number,1], g_mag_star[number,2],star_index,color='red')
    
    #plt.text(positions,star_index,color='red')
    plt.xticks(color="None")
    plt.yticks(color="None")
    plt.tick_params(length=0)
    
    pdf = PdfPages(filename_pdf[idx]) #図をpdfに出力
    pdf.savefig()
    pdf.close()
    plt.show()
    clear_output()
"""
###################################################################
####################################  Create new .csv file ###
photometry_table = np.delete(photometry_table,0,0)
columns =["v_mag_meteor", "v_mag_meteor_err", "star_number",\
              "v_band_abs_mag_meteor", "v_band_abs_mag_meteor_error","zero_mag"]
photometry_table = pd.DataFrame(data=photometry_table,columns=columns)
df = pd.concat([df,photometry_table],axis=1)
df.to_csv("simultaneous_event_candidates_table_MU+Tomo-e9_ucac.csv")
#df.to_csv("test9_ucac.csv")
clear_output()
print "Complete!"

Complete!
